In [3]:
# Develop a Python application for managing product inventory using Tkinter for the graphical user
# interface and SQLite for database storage.
import tkinter as tk
import sqlite3 as sql

# Create a connection to the SQLite database

def connect_db():
    conn = sql.connect('inventory.db')   # Connect to the SQLite database
    c = conn.cursor()
    # Create a table named 'products' if it doesn't already exist
    c.execute('''CREATE TABLE IF NOT EXISTS products
                (id INTEGER PRIMARY KEY, 
                name TEXT NOT NULL
                ,description TEXT
                ,quantity INTEGER NOT NULL
                ,price REAL NOT NULL
                ) ''')
    conn.commit() # save the changes
    conn.close()

connect_db()
            


In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3

# Connect to the SQLite database and create the 'products' table if it doesn't exist
def connect_db():
    conn = sqlite3.connect('inventory.db')
    c = conn.cursor()
    
    # Create a table for products if it doesn't already exist
    c.execute('''CREATE TABLE IF NOT EXISTS products (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    description TEXT,
                    quantity INTEGER NOT NULL,
                    price REAL NOT NULL
                )''')
    conn.commit()
    conn.close()

# Function to add a product to the database
def add_product():
    name = name_entry.get()
    description = description_entry.get()
    quantity = quantity_entry.get()
    price = price_entry.get()

    # Input validation
    if not name or not quantity.isdigit() or not price.replace('.', '', 1).isdigit():
        messagebox.showerror("Input Error", "Please fill all fields with valid data.")
        return

    conn = sqlite3.connect('inventory.db')
    c = conn.cursor()

    # Insert product into the products table
    c.execute("INSERT INTO products (name, description, quantity, price) VALUES (?, ?, ?, ?)",
              (name, description, int(quantity), float(price)))
    conn.commit()
    conn.close()
    
    messagebox.showinfo("Success", "Product added successfully.")
    view_products()  # Refresh the view to show the newly added product
    clear_fields()

# Function to update a product's details
def update_product():
    name = name_entry.get()
    description = description_entry.get()
    quantity = quantity_entry.get()
    price = price_entry.get()

    if not name or not quantity.isdigit() or not price.replace('.', '', 1).isdigit():
        messagebox.showerror("Input Error", "Please fill all fields with valid data.")
        return

    selected_product_id = get_selected_product_id()
    if not selected_product_id:
        messagebox.showerror("Selection Error", "Please select a product to update.")
        return

    conn = sqlite3.connect('inventory.db')
    c = conn.cursor()

    c.execute("UPDATE products SET name = ?, description = ?, quantity = ?, price = ? WHERE id = ?",
              (name, description, int(quantity), float(price), selected_product_id))
    conn.commit()
    conn.close()
    
    messagebox.showinfo("Success", "Product updated successfully.")
    view_products()
    clear_fields()

# Function to delete a product from the database
def delete_product():
    selected_product_id = get_selected_product_id()
    if not selected_product_id:
        messagebox.showerror("Selection Error", "Please select a product to delete.")
        return

    conn = sqlite3.connect('inventory.db')
    c = conn.cursor()

    c.execute("DELETE FROM products WHERE id = ?", (selected_product_id,))
    conn.commit()
    conn.close()

    messagebox.showinfo("Success", "Product deleted successfully.")
    view_products()
    clear_fields()

# Function to get the selected product's ID (for update and delete operations)
def get_selected_product_id():
    try:
        selected_text = product_text.get(tk.SEL_FIRST, tk.SEL_LAST).strip()
        selected_id = int(selected_text.split("|")[0].split(":")[1].strip())
        return selected_id
    except:
        return None

# Function to view all products in the Text widget
def view_products():
    conn = sqlite3.connect('inventory.db')
    c = conn.cursor()

    c.execute("SELECT * FROM products")
    rows = c.fetchall()
    conn.close()

    product_text.delete(1.0, tk.END)

    for row in rows:
        product_text.insert(tk.END, f"ID: {row[0]} | Name: {row[1]} | Description: {row[2]} | Quantity: {row[3]} | Price: {row[4]}\n")

    status_label.config(text=f"Total Products: {len(rows)}")

# Function to clear input fields after adding or updating a product
def clear_fields():
    name_entry.delete(0, tk.END)
    description_entry.delete(0, tk.END)
    quantity_entry.delete(0, tk.END)
    price_entry.delete(0, tk.END)

# Function to sort products
def sort_products(order_by="name", ascending=True):
    conn = sqlite3.connect('inventory.db')
    c = conn.cursor()

    order = "ASC" if ascending else "DESC"
    
    c.execute(f"SELECT * FROM products ORDER BY {order_by} {order}")
    rows = c.fetchall()
    conn.close()

    product_text.delete(1.0, tk.END)
    for row in rows:
        product_text.insert(tk.END, f"ID: {row[0]} | Name: {row[1]} | Description: {row[2]} | Quantity: {row[3]} | Price: {row[4]}\n")

    status_label.config(text=f"Total Products: {len(rows)}")

def on_sort_selection(event):
    selected_sort_option = sort_combobox.get().lower()
    sort_products(order_by=selected_sort_option)

# Main window setup
def create_gui():
    global name_entry, description_entry, quantity_entry, price_entry, product_text, sort_combobox, status_label

    root = tk.Tk()
    root.title("Inventory Management System")
    root.geometry("600x400")

    # Frame for product entry
    entry_frame = tk.Frame(root)
    entry_frame.pack(pady=10)

    tk.Label(entry_frame, text="Name:").grid(row=0, column=0, padx=5, pady=5)
    name_entry = tk.Entry(entry_frame)
    name_entry.grid(row=0, column=1, padx=5, pady=5)

    tk.Label(entry_frame, text="Description:").grid(row=1, column=0, padx=5, pady=5)
    description_entry = tk.Entry(entry_frame)
    description_entry.grid(row=1, column=1, padx=5, pady=5)

    tk.Label(entry_frame, text="Quantity:").grid(row=2, column=0, padx=5, pady=5)
    quantity_entry = tk.Entry(entry_frame)
    quantity_entry.grid(row=2, column=1, padx=5, pady=5)

    tk.Label(entry_frame, text="Price:").grid(row=3, column=0, padx=5, pady=5)
    price_entry = tk.Entry(entry_frame)
    price_entry.grid(row=3, column=1, padx=5, pady=5)

    # Buttons for Add, Update, Delete
    button_frame = tk.Frame(root)
    button_frame.pack(pady=10)

    add_button = tk.Button(button_frame, text="Add", width=10, command=add_product)
    add_button.grid(row=0, column=0, padx=5)

    update_button = tk.Button(button_frame, text="Update", width=10, command=update_product)
    update_button.grid(row=0, column=1, padx=5)

    delete_button = tk.Button(button_frame, text="Delete", width=10, command=delete_product)
    delete_button.grid(row=0, column=2, padx=5)

    # Text widget to display product details with a Scrollbar
    text_frame = tk.Frame(root)
    text_frame.pack(pady=10)

    product_text = tk.Text(text_frame, height=10, width=70)
    product_text.grid(row=0, column=0)

    scrollbar = tk.Scrollbar(text_frame, command=product_text.yview)
    scrollbar.grid(row=0, column=1, sticky='ns')
    product_text.config(yscrollcommand=scrollbar.set)

    # Combobox for sorting options
    sort_label = tk.Label(root, text="Sort by:")
    sort_label.pack(pady=5)

    sort_combobox = ttk.Combobox(root, values=["Name", "Quantity", "Price"])
    sort_combobox.pack(pady=5)
    sort_combobox.bind("<<ComboboxSelected>>", on_sort_selection)

    # Status bar to display the total number of products
    status_label = tk.Label(root, text="Total Products: 0", bd=1, relief=tk.SUNKEN, anchor='w')
    status_label.pack(side=tk.BOTTOM, fill=tk.X)

    view_products()  # Show products when the application starts
    root.mainloop()

# Set up the database and start the GUI
connect_db()
create_gui()

: 